# Opinions and Gaze: Data Analysis (Step 3 of 3)

This Jupyter notebook contains the data analysis for 
for "Seeing the other side: Conflict and controversy 
increase gaze coordination" (Paxton, Dale, & Richardson, 
*in preparation*).

This notebook is the **last of three** notebooks for the 
"Opinions and Gaze" project. This must be run **after** 
the `oag-data_cleaning.ipynb` and `oag-data_processing.ipynb`
files.

To run this file from scratch, you will need:
* `data/04-analysis_dataframes`: Directory of analysis- and
    plotting-ready dataframes, produced by `oag-data_processing.ipynb`.
    * `oag-plotting_df.csv`: Dataframe of real and baseline data.
* `supplementary-code/`: Directory of additional functions and global
    variables.

**Note**: Due to data sensitivity (per the Institutional 
Review Board of the University of California, Merced), 
only researchers from ICPSR member institutions may access
study data through the approved link.

## Variable key

Linear lag (continuous): `ot1`

Quadratic lag (continuous): `ot2`

Opinion congruence (factor, contrast-coded):
- Agreement: `agree = .5`
- Disagreement: `agree = -.5`

Topic class (factor, contrast-coded):
* Mixed-view: `viewtype = .5`
* Dominant-view: `viewtype = -.5`

Data type (factor, contrast-coded):
* Real data: `data = .5`
* Baseline (shuffled) data: `data = -.5`

## Table of contents

* [Preliminaries](#Preliminaries)
    - [Import data and convert factors](#Import-data-and-convert-factors)
* [Descriptive statistics](#Descriptive-statistics)
    - [Derive listener segment statistics](#Derive-listener-segment-statistics)
    - [Derive listener demographic statistics](#Derive-listener-demographic-statistics)
* [Plotting](#Plotting)
* [Data analysis](#Data-analysis)
    - [Plot-level analysis](#Plot-level-analysis)
    - [Planned analyses](#Planned-analyses)
    - [Exploratory analyses](#Exploratory-analyses)
    - [Baseline comparisons](#Baseline-comparisons)

**Written by**: A. Paxton (University of California, Berkeley)

**Date last modified**: 11 April 2018

***

# Preliminaries

In [1]:
# clear the space
rm(list=ls())

In [2]:
# read in the needed files and functions
source('../supplementary-code/libraries_and_functions-oag.r')

Loading required package: crqa
Loading required package: Matrix
Warning message:
“package ‘Matrix’ was built under R version 3.3.2”Loading required package: tseriesChaos
Loading required package: deSolve
Loading required package: fields
Warning message:
“package ‘fields’ was built under R version 3.3.2”Loading required package: spam
Loading required package: grid
Spam version 1.4-0 (2016-08-29) is loaded.
Type 'help( Spam)' or 'demo( spam)' for a short introduction 
and overview of this package.
Help for individual functions is also obtained by adding the
suffix '.spam' to the function name, e.g. 'help( chol.spam)'.

Attaching package: ‘spam’

The following objects are masked from ‘package:base’:

    backsolve, forwardsolve

Loading required package: maps
Loading required package: plot3D
Loading required package: pracma
Warning message:
“package ‘pracma’ was built under R version 3.3.2”
Attaching package: ‘pracma’

The following object is masked from ‘package:deSolve’:

    rk4

The f

## Import data and convert factors

In [3]:
# read in plotting dataframe
plotting_file = file.path(analysis_data_path,
                          'oag-plotting_df.csv')
plotting_df = read.csv(plotting_file,
                       sep=",", header=TRUE)

In [4]:
# create a real-data analysis dataframe
analysis_real_df = plotting_df %>%

    # filter out baseline
    dplyr::filter(data==max(data)) %>%

    # reorder agreement for plotting
    mutate(agree = factor(agree, levels=c(.5, -.5))) %>%

    # factorize everything else
    mutate_at(funs(factor), 
              .vars = factor_variables[!factor_variables %in% c('agree',
                                                                'data')]) %>%

    # convert recurrence to normal distribution 
    mutate(r = Gaussianize(r,
                           type='s'))

Warning message:
“package ‘bindrcpp’ was built under R version 3.3.2”Warning message:
“package ‘lamW’ was built under R version 3.3.2”

In [5]:
# create a real-data plotting dataframe
plotting_real_df = plotting_df %>%

    # filter out baseline
    dplyr::filter(data==max(data)) %>%

    # reorder agreement for plotting
    mutate(agree = factor(agree, levels=c(.5, -.5))) %>%

    # factorize everything else
    mutate_at(funs(factor), 
              .vars = factor_variables[!factor_variables %in% c('agree',
                                                                'data')])

In [6]:
# create a baseline-data analysis dataframe
analysis_baseline_df = plotting_df %>%

    # filter out baseline
    dplyr::filter(data==min(data)) %>%

    # reorder agreement for plotting
    mutate(agree = factor(agree, levels=c(.5, -.5))) %>%

    # factorize everything else
    mutate_at(funs(factor), 
              .vars = factor_variables[!factor_variables %in% c('agree',
                                                                'data')]) %>%
    # convert recurrence to normal distribution 
    mutate(r = Gaussianize(r,
                           type='s')) 

In [7]:
# create a baseline-data plotting dataframe
plotting_baseline_df = plotting_df %>%

    # filter out baseline
    dplyr::filter(data==min(data)) %>%

    # reorder agreement for plotting
    mutate(agree = factor(agree, levels=c(.5, -.5))) %>%

    # factorize everything else
    mutate_at(funs(factor), 
              .vars = factor_variables[!factor_variables %in% c('agree',
                                                                'data')])

In [8]:
# factorize the comparison plotting dataframe
analysis_joint_df = plotting_df %>%

    # reorder agreement for plotting
    mutate(agree = factor(agree, levels=c(.5 ,-.5))) %>%

    # factorize everything else
    mutate_at(funs(factor), 
              .vars = factor_variables[!factor_variables %in% 'agree']) %>%

    # convert recurrence to normal distribution 
    mutate(r = Gaussianize(r,
                           type='s'))

In [9]:
# factorize the comparison plotting dataframe
plotting_df = plotting_df %>%
    
    # reorder agreement for plotting
    mutate(agree = factor(agree, levels=c(.5 ,-.5))) %>%

    # factorize everything else
    mutate_at(funs(factor), 
              .vars = factor_variables[!factor_variables %in% 'agree'])

***

# Descriptive statistics

This section produces basic descriptive statistics 
about individual speakers' segments and about listeners'
demographic data.

## Derive listener segment statistics

### Total number of unique listeners with any recorded data

Listeners who successfully had any recorded data 
will have had a survey saved with their data. We count those
files to show how many listeners had any data recorded.

How many unique **gaze files** do we have?

In [10]:
gaze_file_names = list.files('../data/01-input/listener-gaze-raw',
                             recursive=FALSE)
gaze_participants = str_extract_all(gaze_file_names,
                                    '\\d{5}')
gaze_participants = lapply(gaze_participants, trimws)
length(unique(gaze_participants))

[1] 68

How many unique **questionnaire files** do we have?

In [11]:
questionnaire_file_names = list.files('../data/01-input/listener-responses-raw',
                                      recursive=FALSE,
                                     pattern='*.tsv')
questionnaire_participants = str_extract_all(questionnaire_file_names,
                                             '\\d{5}')
questionnaire_participants = lapply(questionnaire_participants, trimws)
length(unique(questionnaire_participants))

[1] 68

How many **unique participants with gaze and/or questionnaire
files** do we have?

In [12]:
questions_or_gaze_participants = c(unique(gaze_participants),
                                   unique(questionnaire_participants))
length(unique(questions_or_gaze_participants))

[1] 68

### Unfiltered listeners and missing data by segment

Considering all listeners with any recorded data (i.e.,
any listener with _at least some_ gaze data tracked 
during _at least one_ of the segments and with _some_
included metadata) and all segments with at any recorded
data (i.e., no individual listeners' segments in which
no samples were recorded):
1. how many listeners do we have overall,
1. how many listeners do we have per segment, 
1. and what proportion of data are missing per segment?

In [13]:
# load in the missing data table
missing_data_filename = file.path(processed_data_path,
                                  'listener-missing_data.csv')
missing_data = read.table(missing_data_filename,
                          sep=',',
                          header=TRUE)

In [14]:
unfiltered_listeners = missing_data %>% ungroup() %>%
    dplyr::filter(is.na(r_event))

In [15]:
unfiltered_stats = unfiltered_listeners %>% ungroup() %>%
    group_by(topic, side) %>%
    summarise(unique_listeners = n(),
              proportion_missing = round(mean(proportion),
                                        3))

In [16]:
unfiltered_stats

topic,side,unique_listeners,proportion_missing
abortion,only,58,0.376
death-penalty,against,58,0.356
death-penalty,for,57,0.366
drinking-age,for,55,0.314
gay-marriage,only,59,0.308
junk-food-tax,against,55,0.379
junk-food-tax,for,58,0.336
legal-marijuana,only,58,0.357
tax-rich,only,42,0.360


**How many unique listeners were included in the 
unfiltered dataset**?

In [17]:
length(unique(unfiltered_listeners$listener))

[1] 64

On average, **how many listeners were included on each
 segment in the unfiltered dataset**?

In [18]:
round(mean(unfiltered_stats$unique_listeners),2)

[1] 55.56

On average, **what proportion of the gaze data was 
missing across the unfiltered dataset**?

In [19]:
round(mean(unfiltered_stats$proportion_missing),3)

[1] 0.35

### Filtered listeners and missing data by segment

Considering just the listeners that we consider to have
usable data for each segment (i.e., no more than 30% 
missing gaze data in the segment):
1. how many listeners do we have overall,
1. how many listeners do we have per segment, 
1. and what proportion of data are missing per segment?

In [20]:
filtered_listeners = missing_data %>% ungroup() %>%
    dplyr::filter(is.na(r_event) & proportion<=.3)

In [21]:
filtered_stats = filtered_listeners %>% ungroup() %>%
    group_by(topic, side) %>%
    summarise(unique_listeners = n(),
              proportion_missing = round(mean(proportion),
                                        3))

In [22]:
filtered_stats

topic,side,unique_listeners,proportion_missing
abortion,only,31,0.166
death-penalty,against,33,0.148
death-penalty,for,35,0.156
drinking-age,for,35,0.132
gay-marriage,only,34,0.141
junk-food-tax,against,30,0.156
junk-food-tax,for,32,0.123
legal-marijuana,only,36,0.153
tax-rich,only,23,0.138


**How many unique listeners are in the filtered dataset**?

In [23]:
length(unique(filtered_listeners$listener))

[1] 51

On average, **how many listeners were included on each
 segment in the filtered dataset**?

In [24]:
round(mean(filtered_stats$unique_listeners),2)

[1] 32.11

On average, **what proportion of the gaze data was 
missing in the filtered dataset**?

In [25]:
round(mean(filtered_stats$proportion_missing),3)

[1] 0.146

### Filtered segments and missing data by listener

In [26]:
filtered_segments = filtered_listeners %>% ungroup() %>%
    group_by(listener) %>%
    summarise(segments = n(),
              proportion_missing = mean(proportion))

On average, **how many segments does each listener 
contribute to the filtered dataset**?

In [27]:
round(mean(filtered_segments$segments),2)

[1] 5.67

On average, **what proportion of missing data does each listener 
have for segments included in the filtered dataset**?

In [28]:
round(mean(filtered_segments$proportion_missing),3)

[1] 0.169

### Discarded listeners

How many listeners were discarded because they were
**missing required questionnaire or opinion** data?

In [29]:
# create dataframe for missing files
missing_metadata_participants = data.frame()

In [30]:
# load in missing opinion dataframe, if it exists
missing_opinion_filename = file.path(processed_data_path,
                                     'listener-missing_opinions.csv')
if (file.exists(missing_opinion_filename)){

    # add reason for missing
    missing_opinions = read.table(missing_opinion_filename,
                                  sep=",",
                                  header=TRUE) %>%
        mutate(reason = 'missing_opinions')
    
    # append to dataframe
    missing_metadata_participants = rbind.data.frame(missing_metadata_participants,
                                                     missing_opinions)
}

In [31]:
# load in missing questionnaire dataframe, if it exists
missing_questionnaire_filename = file.path(processed_data_path,
                                           'listener-missing_questionnaire.csv')
if (file.exists(missing_questionnaire_filename)){

    # add reason for missing
    missing_questionnaire = read.table(missing_questionnaire_filename,
                                       sep=",",
                                       header=TRUE)
    
    # append to dataframe
    missing_metadata_participants = rbind.data.frame(missing_metadata_participants,
                                                     missing_questionnaire)
}

In [32]:
length(unique(missing_metadata_participants$listener))

[1] 1

How many listeners were completely discarded from the dataset
due to having **more than 30% missing data in all trials?**

In [33]:
discarded_listeners = missing_data %>% ungroup() %>%
    dplyr::filter(is.na(r_event)) %>%
    dplyr::filter(!(listener %in% unique(filtered_segments$listener)))

In [34]:
length(unique(discarded_listeners$listener))

[1] 13

How many listeners were discarded due to **other equipment error**?

In [35]:
discarded_participants = c(unique(missing_metadata_participants$listener),
                           unique(discarded_listeners$listener))

In [36]:
recruited_participants = unique(questions_or_gaze_participants)

In [37]:
included_participants = unique(filtered_listeners$listener)

In [38]:
length(recruited_participants) - 
    length(discarded_participants) -
        length(included_participants)

[1] 3

## Derive listener demographic statistics

**Note**: All numeric categories were derived alphabetically.

In [39]:
# read in dataset
demographics_data = plotting_real_df %>% ungroup() %>%

    # keep only the columns we need
    dplyr::select(one_of(crqa_questionnaire_columns),
           -topic_and_side,
           -agree) %>%

    # only one line per participant
    distinct()

Warning message:
“Unknown variables: `topic`, `side`”

### What is the self-reported **gender distribution**?

*Note: Participants were asked about gender but 
were provided with sex categories.*

In [41]:
gender_data = demographics_data %>% ungroup() %>%
    group_by(gender) %>%
    summarise(gender_counts = n(),
              gender_proportion = round((n()/nrow(demographics_data)),
                                        3))

In [42]:
# female = lower, male = higher
gender_data

gender,gender_counts,gender_proportion
-0.352941176470588,32,0.627
0.647058823529412,19,0.373


### What is the self-reported **mean age**?

In [43]:
age = demographics_data %>%
    dplyr::select(age) %>%
    mutate(age = gsub("[^0-9.]", "", age)) %>%
    dplyr::filter(age != '') %>%
    mutate(age = as.numeric(age)) %>%
    .$age

In [44]:
round(mean(age),2)

[1] 19.84

### What is the self-reported **native language** distribution?

In [45]:
native_lang_data = demographics_data %>% ungroup() %>%

    # convert all to lowercase
    mutate(native_lang = tolower(native_lang)) %>%
    mutate(native_lang = trimws(native_lang)) %>%
    
    # get counts
    group_by(native_lang) %>%
    summarise(native_lang_counts = n(),
              native_lang_proportion = round(n()/nrow(demographics_data),
                                            3)) %>%
    ungroup() %>%

    # arrange in descending order
    dplyr::arrange(native_lang)

In [46]:
# English = lowest, Spanish = middle, other = highest
native_lang_data

native_lang,native_lang_counts,native_lang_proportion
-0.792387543252595,19,0.373
0.207612456747405,22,0.431
1.2076124567474,10,0.196


## Derive listener opinion statistics

In [47]:
# read in dataset
opinion_data = plotting_real_df %>% ungroup() %>%

    # keep only the columns we need
    dplyr::select(one_of(crqa_questionnaire_columns),
           topic_and_side,
          -gender,
          -native_lang,
          -age) %>%

    # get one line per topic and side
    distinct()

Warning message:
“Unknown variables: `topic`, `side`”

### What is the **distribution of self-reported agreement** of listeners with the speaker, regardless of topic and side?

In [48]:
agree_bias = opinion_data %>% ungroup() %>%
    group_by(agree) %>%
    summarise(agree_counts = n(),
              agree_proportions = round(n()/nrow(opinion_data),
                                       3))

In [49]:
# agree = .5, disagree = -.5
agree_bias

agree,agree_counts,agree_proportions
0.5,192,0.664
-0.5,97,0.336


***

# Plotting

## Distribution of opinion congruence by topic class

In [50]:
# prepare for histogram
agreement_plots = plotting_df %>% ungroup() %>%

    # get just one line per listener per topic
    dplyr::select(speaker, listener, topic_and_side, agree, viewtype) %>%
    group_by(speaker, listener, topic_and_side, agree, viewtype) %>%
    distinct() %>%
    ungroup() %>%

    # group for counts
    group_by(agree, viewtype) %>%
    summarise(counts = n()) %>%
    ungroup() %>%

    # convert counts to proportion
    mutate(proportion = counts/sum(counts))

In [51]:
# create the plot
agreement_distribution = ggplot(data=agreement_plots,
                                aes(x=as.factor(agree),
                                    y=proportion,
                                    fill=as.factor(viewtype)),
                               labeller = agree_labeller) +
    geom_bar(stat="identity",
             position='dodge')+
    scale_fill_manual(name="Opinion congruence",
                      breaks=c(.5,-.5),
                      labels=c("Agree", "Disagree"),
                      values=c("#b2182b", "#67a9cf")) +
    scale_x_discrete(labels=c("Mixed-view","Dominant-view")) +
    theme(legend.position='bottom') +
    xlab("Topic class") +
    ylab("Proportion") +
    ggtitle("Listener agreement by topic class\nand opinion congruence")

In [52]:
# save a high-resolution version
ggsave(plot = agreement_distribution,
       height = 4,
       width = 4,
       filename = '../figures/gca-agreement_viewtype_distribution.jpg')

In [53]:
# save a smaller version of the plot
ggsave(plot = agreement_distribution,
       height = 4,
       width = 4,
       dpi=100,
       filename = '../figures/gca-agreement_viewtype_distribution-inline.jpg')

![](../figures/gca-agreement_viewtype_distribution-inline.jpg)

The plot above breaks down listeners' self-reported
agreement (blue) or disagreement (red) with each segment, along
with whether that segment was part of a mixed- (right)
or dominant-view (left) topic.

## Individual segment plot for all listeners

What do all listeners' individual segments look like?

In [54]:
# plot all individual listeners
all_listener_plot = ggplot(plotting_real_df, 
                           aes(x=t, 
                               y=r, 
                               group=topic_and_side, 
                               color=as.factor(agree))) + 
    facet_wrap(~listener) +
    geom_path() +
    scale_color_manual(name="Opinion congruence",
                      labels=c("Agree", "Disagree"),
                      values=c("#67a9cf", "#b2182b")) +
    xlab("Lag (in 10Hz samples)") +
    ylab("Recurrence (rec)") +
    ggtitle("Gaze coordination between listeners and speakers
by lag and opinion congruence") +
    theme(strip.text.x = element_blank(),
          strip.background = element_rect(colour="white", 
                                          fill="white"),
      legend.position='bottom')

In [55]:
# save a high-resolution version
ggsave(plot = all_listener_plot,
       height = 7,
       width = 7,
       filename = '../figures/gca-individual_trials.jpg')

In [56]:
# save an inline display version
ggsave(plot = all_listener_plot,
       height = 7,
       width = 7,
       dpi=100,
       filename = '../figures/gca-individual_trials-inline.jpg')

![](../figures/gca-individual_trials-inline.jpg)

Each panel above presents all usable data from a single
listener. Each line is a single diagonal recurrence profile
between the listener and the speaker for a single audio
segment. Lines are color-coded according to whether the 
listener rated having agreed with (blue) or disagreed with
(red) the speaker after hearing their segment.

## Gaze coordination by opinion congruence and lag

How does speaker-listener gaze coordination look when 
considering listeners' disagreement versus agreement?

In [57]:
# plot recurrence by agreement only
r_by_agreement = ggplot(data=plotting_real_df,
                        aes(x=t, 
                            y=r, 
                            color=agree,
                            group=agree)) + 
    # plot mean DRP curves
    geom_smooth(method="loess",
                se=TRUE) +

    # add in lines for raw means
    geom_line(data = plotting_real_df %>%
                      group_by(t, agree) %>%
                      summarise(r = mean(r)),
              aes(x = t,
                  y = r,
                  color = agree,
                  group = agree)) +

    # set color by agreement
    scale_color_manual(name="Opinion congruence",
                       breaks=c("1","0"),
                       labels=c("Agree","Disagree"),
                       values=c("#67a9cf","#b2182b")) + 

    # set labels
    xlab("Lag (in 10Hz samples)") +
    ylab("Recurrence (rec)") +
    ggtitle("Gaze coordination by lag
and opinion congruence") +
    theme(strip.background = element_rect(colour="white", 
                                          fill="white"),
          legend.position='bottom') 

In [58]:
# save a high-resolution version
ggsave(plot = r_by_agreement,
       height = 5,
       width = 4,
       filename = '../figures/gca-main_interaction_plot.jpg')

In [59]:
# save an inline display version
ggsave(plot = r_by_agreement,
       height = 5,
       width = 4,
       dpi=100,
       filename = '../figures/gca-main_interaction_plot-inline.jpg')

![](../figures/gca-main_interaction_plot-inline.jpg)

The figure above shows the aggregated diagonal 
recurrence profile (DRP) for the planned analysis,
predicting recurrence (*rec*) with opinion
congruence (listener agreement [blue] versus disagreement
[red] with the speaker in each segment) and lag.

## Gaze coordination by opinion congruence, topic class, and lag

How does gaze coordination look when considering opinion 
congruence (agreement versus disagreement) and topic
class (dominant- versus mixed-view topics)?

In [60]:
# plot recurrence by agreement and viewtype
r_by_agreement_and_viewtype = ggplot(data=plotting_real_df %>%
                                         mutate(agree = ifelse(agree==-.5,
                                                               "Disagree",
                                                               "Agree")), 
                                     aes(x=t, 
                                         y=r, 
                                         color=viewtype,
                                         group=viewtype)) + 

    # add in lines for raw means
    geom_line(data = plotting_real_df %>%
                      mutate(agree = ifelse(agree==-.5,
                                            "Disagree",
                                            "Agree")) %>%
                      group_by(t, agree, viewtype) %>%
                      summarise(r = mean(r)),
              aes(x = t,
                  y = r,
                  color = viewtype,
                  group = viewtype)) +

    # separate by agreement
    facet_wrap(~ agree) +

    # set color by dominant- versus mixed-view
    scale_color_manual(name="Topic class", 
                       breaks=c(-.5, .5),
                       labels=c("Dominant-view", "Mixed-view"),
                       values=c("#d95f02","#7570b3")) + 

    # set labels
    xlab("Lag (in 10Hz samples)") +
    ylab("Recurrence (rec)") +
    ggtitle("Gaze coordination by lag,
opinion congruence, and topic class") +
    theme(strip.background = element_rect(colour="white", 
                                          fill="white"),
          legend.position='bottom') +

    # plot mean DRP curves
    geom_smooth(method="loess",
                se=TRUE)

In [61]:
# save a high-resolution version
ggsave(plot = r_by_agreement_and_viewtype,
       height = 5,
       width = 4,
       filename = '../figures/gca-exploratory_interaction_plot.jpg')

In [62]:
# save an inline display version
ggsave(plot = r_by_agreement_and_viewtype,
       height = 5,
       width = 4,
       dpi=100,
       filename = '../figures/gca-exploratory_interaction_plot-inline.jpg')

![](../figures/gca-exploratory_interaction_plot-inline.jpg)

The figure above shows the aggregated diagonal 
recurrence profile (DRP) for the exploratory analysis,
predicting recurrence (*rec*) with opinion
congruence (listener agreement [left panel] versus 
disagreement [right panel] with the speaker in each segment),
topic class (dominant-view [orange] or mixed-view [blue]), and lag.

***

# Data analysis

## Plot-level analysis

In [63]:
# create subsets for plot-level metric analysis
one_liner = plotting_real_df %>%
    dplyr::filter(t==0) %>%

    # shift lag to account for window
    mutate(maxlag = maxlag-win_size-1)

# separate for agreement and disagreement
one_liner_agree = one_liner %>%
    dplyr::filter(agree==.5)
one_liner_disagree = one_liner %>%
    dplyr::filter(agree==-.5)

### Maximum recurrence

Is maximum recurrence different from 0? Yes.

In [64]:
t.test(one_liner$maxrec)


	One Sample t-test

data:  one_liner$maxrec
t = 33.312, df = 288, p-value < 0.00000000000000022
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 0.08479688 0.09544670
sample estimates:
 mean of x 
0.09012179 


Are the maximum recurrence values for both
agreement and disagreement different from 0?
Yes for each.

In [65]:
t.test(one_liner_agree$maxrec)


	One Sample t-test

data:  one_liner_agree$maxrec
t = 27.489, df = 191, p-value < 0.00000000000000022
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 0.07753789 0.08952539
sample estimates:
 mean of x 
0.08353164 


In [66]:
t.test(one_liner_disagree$maxrec)


	One Sample t-test

data:  one_liner_disagree$maxrec
t = 20.083, df = 96, p-value < 0.00000000000000022
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 0.09296928 0.11336314
sample estimates:
mean of x 
0.1031662 


Does maximum recurrence differ between
agreement and disagreement? No.

In [67]:
lm.maxrec.agree = lmer(maxrec ~ agree + 
                       (1 + agree | listener) + 
                       (1 + agree | topic_and_side), 
                       data = one_liner)
pander_lme(lm.maxrec.agree)



|      &nbsp;       |  Estimate  |  Std..Error  |  t.value  |   p    |  sig  |
|:-----------------:|:----------:|:------------:|:---------:|:------:|:-----:|
|  **(Intercept)**  |  0.08872   |   0.01339    |   6.623   | 0.0001 |  ***  |
|   **agree-0.5**   | -0.001505  |   0.005324   |  -0.2828  |  0.78  |       |



What is the mean maximum recurrence for agreement
and for disagreement?

In [68]:
print(paste0('Mean maximum recurrence for agreement: ', 
             round(mean(one_liner_agree$maxrec), 3)))

[1] "Mean maximum recurrence for agreement: 0.084"


In [69]:
print(paste0('Mean maximum recurrence for disagreement: ',
             round(mean(one_liner_disagree$maxrec), 3)))

[1] "Mean maximum recurrence for disagreement: 0.103"


### Maximum lag

Is maxlag different from 0 overall? No.

In [70]:
t.test(one_liner$maxlag)


	One Sample t-test

data:  one_liner$maxlag
t = 0.33115, df = 288, p-value = 0.7408
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 -1.847479  2.594884
sample estimates:
mean of x 
0.3737024 


Is maximum lag for each agreement group different 
from 0? No for each.

In [71]:
t.test(one_liner_agree$maxlag)


	One Sample t-test

data:  one_liner_agree$maxlag
t = 0.03414, df = 191, p-value = 0.9728
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 -2.661337  2.755087
sample estimates:
mean of x 
 0.046875 


In [72]:
t.test(one_liner_disagree$maxlag)


	One Sample t-test

data:  one_liner_disagree$maxlag
t = 0.51336, df = 96, p-value = 0.6089
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 -2.925723  4.966960
sample estimates:
mean of x 
 1.020619 


Does maximum lag differ by agreement or disagreement?
No.

In [73]:
lm.maxlag.agree = lmer(maxlag ~ agree + 
                       (1 + agree | listener) + 
                       (1 | topic_and_side), 
                       data = one_liner)
pander_lme(lm.maxlag.agree)



|      &nbsp;       |  Estimate  |  Std..Error  |  t.value  |  p   |  sig  |
|:-----------------:|:----------:|:------------:|:---------:|:----:|:-----:|
|  **(Intercept)**  |  0.06094   |    1.426     |  0.04273  | 0.97 |       |
|   **agree-0.5**   |   0.7486   |    2.394     |  0.3127   | 0.75 |       |



What is the mean maximum lag for agreement and
for disagreement?

In [74]:
print(paste0('Mean maximum lag for agreement: ',
             round(mean(one_liner_agree$maxlag), 2),
             ' samples (',
             round(mean(one_liner_agree$maxlag)/10, 2),
             ' sec)'))

[1] "Mean maximum lag for agreement: 0.05 samples (0 sec)"


In [75]:
print(paste0('Mean maximum lag for disagreement: ',
             round(mean(one_liner_disagree$maxlag), 2),
             ' samples (',
             round(mean(one_liner_disagree$maxlag)/10, 2),
             ' sec)'))

[1] "Mean maximum lag for disagreement: 1.02 samples (0.1 sec)"


## Planned analyses

Here, we'll test the relations among recurrence (`r`), 
linear lag (`ot1`), quadratic lag (`ot2`), and opinion congruence
(`agree`) that we had anticipated testing at the outset of the study.

### Two-way interaction model

First, we'll model the data using all main terms and up
to two-way interaction terms.

In [76]:
planned_model_twowayint = lmer(r ~ ot1 + ot2 + agree +
                               ot1:agree + ot2:agree + ot1:ot2 +
                               (1 + ot1 + ot2 + agree | listener) + 
                               (1 + ot1 + ot2 + agree | topic_and_side), 
                               data=analysis_real_df)

In [77]:
pander_lme(planned_model_twowayint)



|       &nbsp;        |  Estimate  |  Std..Error  |  t.value  |   p    |  sig  |
|:-------------------:|:----------:|:------------:|:---------:|:------:|:-----:|
|   **(Intercept)**   |  0.06542   |   0.01323    |   4.946   | 0.0001 |  ***  |
|       **ot1**       | -0.004661  |   0.00855    |  -0.5452  |  0.59  |       |
|       **ot2**       | -0.001987  |   0.006938   |  -0.2864  |  0.78  |       |
|    **agree-0.5**    |  -0.01107  |   0.004569   |  -2.422   | 0.015  |   *   |
|  **ot1:agree-0.5**  |  0.004621  |   0.002023   |   2.284   | 0.022  |   *   |
|  **ot2:agree-0.5**  |  0.005464  |   0.001999   |   2.733   | 0.006  |  **   |
|     **ot1:ot2**     | 0.0009637  |   0.007659   |  0.1258   |  0.9   |       |



### Full interaction model

Next, let's include the main terms and all possible 
interaction terms.

In [78]:
planned_model_allint = lmer(r ~ ot1 * ot2 * agree +
                            (1 + ot1 + ot2 + agree | listener) + 
                            (1 + ot1 + ot2 + agree | topic_and_side), 
                            data=analysis_real_df)

In [79]:
pander_lme(planned_model_allint)



|         &nbsp;          |  Estimate  |  Std..Error  |  t.value  |   p    |  sig  |
|:-----------------------:|:----------:|:------------:|:---------:|:------:|:-----:|
|     **(Intercept)**     |  0.07311   |   0.01386    |   5.276   | 0.0001 |  ***  |
|         **ot1**         |  -0.01449  |   0.01005    |  -1.442   | 0.149  |       |
|         **ot2**         | -0.009901  |   0.008139   |  -1.217   | 0.224  |       |
|      **agree-0.5**      |  -0.03397  |   0.01314    |  -2.586   |  0.01  |   *   |
|       **ot1:ot2**       |  0.01109   |   0.009396   |   1.18    | 0.238  |       |
|    **ot1:agree-0.5**    |  0.03391   |   0.01588    |   2.135   | 0.033  |   *   |
|    **ot2:agree-0.5**    |  0.02904   |   0.01284    |   2.262   | 0.024  |   *   |
|  **ot1:ot2:agree-0.5**  |  -0.03016  |   0.01622    |   -1.86   | 0.063  |   .   |



### Model comparison

Which model better accounts for the data?

In [80]:
anova(planned_model_twowayint,
      planned_model_allint)

refitting model(s) with ML (instead of REML)


,Df,AIC,BIC,logLik,deviance,Chisq,Chi Df,Pr(>Chisq)
planned_model_twowayint,28,-98119.01,-97901.24,49087.50,-98175.01,NA,NA,NA
planned_model_allint,29,-98120.46,-97894.92,49089.23,-98178.46,3.458249,1,0.06293694


The model with all possible interaction terms trends toward
but does not significantly perform better than the model
with only two-way interaction terms.

## Exploratory analyses

Next, we'll look into the relations among recurrence (`r`), 
linear lag (`ot1`), quadratic lag (`ot2`), opinion congruence
(`agree`), and topic class (`viewtype`) -- a new variable 
that we had not expected to explore at the outset of the study.

### Two-way interaction model

First, we'll model the data using all main terms and up
to two-way interaction terms.

In [81]:
exploratory_model_twowayint = lmer(r ~ ot1 + ot2 + agree + viewtype +
                                   ot1:agree + ot2:agree +
                                   ot1:viewtype + ot2:viewtype +
                                   agree:viewtype + ot1:ot2 +
                                   (1 + ot1 + ot2 + agree + viewtype | listener) + 
                                   (1 + ot1 + ot2 + agree + viewtype | topic_and_side), 
                                   data=analysis_real_df)

In [82]:
pander_lme(exploratory_model_twowayint)



|           &nbsp;            |  Estimate  |  Std..Error  |  t.value  |   p   |  sig  |
|:---------------------------:|:----------:|:------------:|:---------:|:-----:|:-----:|
|       **(Intercept)**       |  0.04945   |   0.02003    |   2.469   | 0.014 |   *   |
|           **ot1**           | -0.002962  |   0.008594   |  -0.3447  | 0.73  |       |
|           **ot2**           |  0.003951  |   0.007004   |   0.564   | 0.57  |       |
|        **agree-0.5**        |  -0.01357  |   0.006366   |  -2.131   | 0.033 |   *   |
|       **viewtype0.5**       |   0.0302   |   0.02898    |   1.042   |  0.3  |       |
|      **ot1:agree-0.5**      |  0.005014  |   0.001901   |   2.637   | 0.008 |  **   |
|      **ot2:agree-0.5**      |  0.005575  |   0.001867   |   2.987   | 0.003 |  **   |
|     **ot1:viewtype0.5**     |  -0.00354  |   0.004668   |  -0.7585  | 0.45  |       |
|     **ot2:viewtype0.5**     |  -0.01081  |   0.004956   |  -2.181   | 0.029 |   *   |
|  **agree-0.5:viewtype0.5**  

### Full interaction model

Next, let's include the main terms and all possible interaction terms.

In [83]:
exploratory_model_allint = lmer(r ~ ot1 * ot2 * agree * viewtype +
                                (1 + ot1 + ot2 + agree + viewtype | listener) + 
                                (1 + ot1 + ot2 + agree + viewtype | topic_and_side), 
                                data=analysis_real_df)

In [84]:
pander_lme(exploratory_model_allint)



|               &nbsp;                |  Estimate  |  Std..Error  |  t.value  |   p    |  sig  |
|:-----------------------------------:|:----------:|:------------:|:---------:|:------:|:-----:|
|           **(Intercept)**           |  0.03278   |   0.02234    |   1.467   | 0.142  |       |
|               **ot1**               |  0.01771   |   0.01321    |   1.341   |  0.18  |       |
|               **ot2**               |   0.0199   |    0.0107    |   1.861   | 0.063  |   .   |
|            **agree-0.5**            | -0.009714  |   0.02094    |  -0.4638  |  0.64  |       |
|           **viewtype0.5**           |  0.07826   |   0.03396    |   2.305   | 0.021  |   *   |
|             **ot1:ot2**             |  -0.01869  |   0.01257    |  -1.487   | 0.137  |       |
|          **ot1:agree-0.5**          |  0.002696  |   0.02545    |  0.1059   |  0.92  |       |
|          **ot2:agree-0.5**          |  0.00655   |   0.02056    |  0.3186   |  0.75  |       |
|         **ot1:viewtype0.5*

### Model comparison

Which model better accounts for the data?

In [85]:
anova(exploratory_model_twowayint,
      exploratory_model_allint)

refitting model(s) with ML (instead of REML)


,Df,AIC,BIC,logLik,deviance,Chisq,Chi Df,Pr(>Chisq)
exploratory_model_twowayint,42,-100049,-99722.38,50066.51,-100133,NA,NA,NA
exploratory_model_allint,47,-100061,-99695.49,50077.51,-100155,21.99167,5,0.00052551


Here, the full model accounts for the data
significantly better than the model that 
includes only limited interaction terms.

### Post-hoc analyses

Given the possible differences by opinion congruence and 
topic class, let's dig a bit more deeply into the exploratory
model. What's actually driving the significant effects that
we see?

In [86]:
# separate dataframes by type
topic_dfs = split(analysis_real_df,
                  analysis_real_df$viewtype)
dominant_df = topic_dfs$`-0.5`
mixed_df = topic_dfs$`0.5`

**Dominant-view model** (`viewtype = -.5`)

In [87]:
exploratory_model_dominant = lmer(r ~ ot1 * ot2 * agree +
                                (1 + ot1 + ot2 + agree | listener) + 
                                (1 + ot1 + ot2 + agree | topic_and_side), 
                                data=dominant_df)

In [88]:
pander_lme(exploratory_model_dominant)



|         &nbsp;          |  Estimate  |  Std..Error  |  t.value  |   p   |  sig  |
|:-----------------------:|:----------:|:------------:|:---------:|:-----:|:-----:|
|     **(Intercept)**     |  0.03087   |   0.01832    |   1.685   | 0.092 |   .   |
|         **ot1**         |  0.01989   |   0.01306    |   1.523   | 0.128 |       |
|         **ot2**         |  0.01926   |   0.01176    |   1.638   | 0.101 |       |
|      **agree-0.5**      |  -0.00482  |   0.02018    |  -0.2388  | 0.81  |       |
|       **ot1:ot2**       |  -0.01869  |   0.01219    |  -1.533   | 0.125 |       |
|    **ot1:agree-0.5**    | 0.0004748  |   0.02471    |  0.01922  | 0.98  |       |
|    **ot2:agree-0.5**    |  0.01001   |   0.01998    |  0.5006   | 0.62  |       |
|  **ot1:ot2:agree-0.5**  | -0.004327  |   0.02521    |  -0.1717  | 0.86  |       |



We find no significant effects for dominant-view topics based on
listener agreement or disagreement.

**Mixed-view model** (`viewtype = .5`)

In [89]:
exploratory_model_mixed = lmer(r ~ ot1 * ot2 * agree +
                                (1 + ot1 + ot2 + agree | listener) + 
                                (1 + ot1 + ot2 + agree | topic_and_side), 
                                data=mixed_df)

In [90]:
pander_lme(exploratory_model_mixed)



|         &nbsp;          |  Estimate  |  Std..Error  |  t.value  |   p    |  sig  |
|:-----------------------:|:----------:|:------------:|:---------:|:------:|:-----:|
|     **(Intercept)**     |   0.1128   |   0.01859    |   6.068   | 0.0001 |  ***  |
|         **ot1**         |  -0.04638  |   0.01318    |  -3.519   | 0.0004 |  ***  |
|         **ot2**         |  -0.03933  |   0.009552   |  -4.118   | 0.0001 |  ***  |
|      **agree-0.5**      |  -0.06297  |   0.01538    |  -4.094   | 0.0001 |  ***  |
|       **ot1:ot2**       |  0.04025   |   0.01173    |   3.431   | 0.001  |  **   |
|    **ot1:agree-0.5**    |  0.06565   |   0.01791    |   3.667   | 0.0002 |  ***  |
|    **ot2:agree-0.5**    |  0.05119   |   0.01446    |   3.54    | 0.0004 |  ***  |
|  **ot1:ot2:agree-0.5**  |  -0.05764  |   0.01827    |  -3.154   | 0.002  |  **   |



The mixed-view topics appear to be doing the majority of the heavy
lifting with these data: All effects here are significant.

## Baseline comparisons

Let's add a term to account for real (`data=.5`) versus baseline 
data (`data=-.5`) to the planned and exploratory models reported
above. In each, we'll again compare models that include only 
up to two-way interactions with those that include all 
possible interaction terms to see which better account for the data.

### Planned analysis

#### Two-way interaction model

In [91]:
planned_model_baseline_twowayint = lmer(r ~ ot1 + ot2 + agree + data +
                                        ot1:agree + ot2:agree + 
                                        ot1:data + ot2:data + 
                                        agree:data + ot1:ot2 +
                                        (1 + agree | listener) + 
                                        (1 | topic_and_side), 
                                        data=analysis_joint_df)

In [92]:
pander_lme(planned_model_baseline_twowayint)



|         &nbsp;          |  Estimate  |  Std..Error  |  t.value  |   p    |  sig  |
|:-----------------------:|:----------:|:------------:|:---------:|:------:|:-----:|
|     **(Intercept)**     |  0.06424   |   0.01197    |   5.367   | 0.0001 |  ***  |
|         **ot1**         | -0.003331  |   0.001919   |  -1.736   | 0.083  |   .   |
|         **ot2**         | -0.002774  |   0.001554   |  -1.785   | 0.074  |   .   |
|      **agree-0.5**      | -0.0003169 |   0.001872   |  -0.1693  |  0.87  |       |
|       **data0.5**       | 0.0003562  |  0.0008942   |  0.3983   |  0.69  |       |
|    **ot1:agree-0.5**    | -0.0008192 |  0.0004651   |  -1.761   | 0.078  |   .   |
|    **ot2:agree-0.5**    |  0.001456  |  0.0004651   |   3.131   | 0.002  |  **   |
|     **ot1:data0.5**     | -0.000742  |  0.0007639   |  -0.9713  |  0.33  |       |
|     **ot2:data0.5**     | -0.0003869 |  0.0007639   |  -0.5065  |  0.61  |       |
|  **agree-0.5:data0.5**  | -0.0004536 |  0.0002071   |   -2.19

#### Full interaction model

In [93]:
planned_model_baseline_allint = lmer(r ~ ot1 * ot2 * agree * data +
                                     (1 + agree | listener) + 
                                     (1 | topic_and_side), 
                                     data=analysis_joint_df)

In [94]:
pander_lme(planned_model_baseline_allint)



|             &nbsp;              |  Estimate  |  Std..Error  |  t.value  |   p    |  sig  |
|:-------------------------------:|:----------:|:------------:|:---------:|:------:|:-----:|
|         **(Intercept)**         |  0.06473   |   0.01203    |   5.381   | 0.0001 |  ***  |
|             **ot1**             | -0.004067  |   0.00246    |  -1.654   | 0.098  |   .   |
|             **ot2**             | -0.003422  |   0.001987   |  -1.722   | 0.085  |   .   |
|          **agree-0.5**          | -0.001308  |   0.003795   |  -0.3447  |  0.73  |       |
|           **data0.5**           |  0.008107  |   0.006429   |   1.261   | 0.207  |       |
|           **ot1:ot2**           |  0.003625  |   0.002515   |   1.441   |  0.15  |       |
|        **ot1:agree-0.5**        | 0.0007918  |   0.004245   |  0.1865   |  0.85  |       |
|        **ot2:agree-0.5**        |  0.002921  |   0.00343    |  0.8516   |  0.39  |       |
|         **ot1:data0.5**         | -0.009389  |   0.008157   |  -1.

#### Model comparison

In [95]:
anova(planned_model_baseline_twowayint,
      planned_model_baseline_allint)

refitting model(s) with ML (instead of REML)


,Df,AIC,BIC,logLik,deviance,Chisq,Chi Df,Pr(>Chisq)
planned_model_baseline_twowayint,16,-1152108,-1151945,576069.8,-1152140,NA,NA,NA
planned_model_baseline_allint,21,-1152123,-1151910,576082.7,-1152165,25.73546,5,0.0001004188


We're significantly better able to maximize log-likelihood 
of the model by including all possible interaction terms, 
rather than just including two-way interactions.

### Exploratory analysis

#### Two-way interaction model

In [96]:
exploratory_model_baseline_twowayint = lmer(r ~ ot1 + ot2 + agree + viewtype + data +
                                            ot1:agree + ot2:agree + 
                                            ot1:viewtype + ot2:viewtype +
                                            ot1:ot2 + agree:viewtype +
                                            ot1:data + ot2:data + 
                                            viewtype:data + agree:data +
                                            (1 + data | listener) + 
                                            (1 + data | topic_and_side), 
                                            data=analysis_joint_df)

In [97]:
pander_lme(exploratory_model_baseline_twowayint)



|           &nbsp;            |   Estimate   |  Std..Error  |  t.value  |   p    |  sig  |
|:---------------------------:|:------------:|:------------:|:---------:|:------:|:-----:|
|       **(Intercept)**       |   0.05171    |   0.01845    |   2.802   | 0.005  |  **   |
|           **ot1**           |  -0.001227   |   0.002058   |  -0.5965  |  0.55  |       |
|           **ot2**           |  -0.002773   |   0.001672   |  -1.658   | 0.097  |   .   |
|        **agree-0.5**        |   0.002929   |  0.0005975   |   4.903   | 0.0001 |  ***  |
|       **viewtype0.5**       |   0.02221    |   0.02462    |  0.9023   |  0.37  |       |
|         **data0.5**         |  0.0005654   |   0.001481   |  0.3817   |  0.7   |       |
|      **ot1:agree-0.5**      | -0.000003684 |  0.0005041   | -0.007308 |  0.99  |       |
|      **ot2:agree-0.5**      |   0.001457   |  0.0005041   |   2.89    | 0.004  |  **   |
|     **ot1:viewtype0.5**     |  -0.004165   |  0.0004809   |   -8.66   | 0.0001 |  *** 

#### Full interaction model

In [98]:
exploratory_model_baseline_allint = lmer(r ~ ot1 * ot2 * agree * viewtype * data +
                                         (1 + data | listener) + 
                                         (1 + data | topic_and_side), 
                                         data=analysis_joint_df)

In [99]:
pander_lme(exploratory_model_baseline_allint)



|                   &nbsp;                    |  Estimate   |  Std..Error  |  t.value  |   p    |  sig  |
|:-------------------------------------------:|:-----------:|:------------:|:---------:|:------:|:-----:|
|               **(Intercept)**               |   0.05214   |   0.01861    |   2.802   | 0.005  |  **   |
|                   **ot1**                   |  -0.000602  |   0.003722   |  -0.1617  |  0.87  |       |
|                   **ot2**                   |  -0.003444  |   0.003007   |  -1.145   |  0.25  |       |
|                **agree-0.5**                | -0.0004907  |   0.006066   | -0.08089  |  0.94  |       |
|               **viewtype0.5**               |   0.02233   |   0.02495    |  0.8951   |  0.37  |       |
|                 **data0.5**                 |  -0.02251   |   0.009794   |  -2.299   | 0.022  |   *   |
|                 **ot1:ot2**                 |  0.002148   |   0.003807   |  0.5643   |  0.57  |       |
|              **ot1:agree-0.5**            

#### Model comparison

In [100]:
anova(exploratory_model_baseline_twowayint,
      exploratory_model_baseline_allint)

refitting model(s) with ML (instead of REML)


,Df,AIC,BIC,logLik,deviance,Chisq,Chi Df,Pr(>Chisq)
exploratory_model_baseline_twowayint,23,-1127827,-1127593,563936.3,-1127873,NA,NA,NA
exploratory_model_baseline_allint,39,-1127911,-1127515,563994.7,-1127989,116.8469,16,0.00000000000000002213207


Again, the model that includes all possible interaction
terms is significantly better able to capture the data
than the model that includes only two-way interaction terms.

# Discussion

Gaze patterns -- that is, the ways in which people look at
things in their environment over time -- can reveal essential
information about attention and understanding during conversation.
When analyzing gaze patterns that are only tied to perception and
information processing and *not* social signaling (i.e., looking
at images while listening to a monologue without the speaker in
the room), previous research has shown listeners who have more similar 
(or more *coordinated*) gaze patterns with speakers better understand
the monologues.

This experiment explores the degree to which the coupling of
information-processing gaze can be affected by the broader
context of the interaction. We designed the experiment to
test specifically listeners' agreement or disagreement with
the speaker, but we unexpectedly found that the data could
shed light on broader social processes around sociopolitical
issues -- namely, whether the issue has been largely settled
in the social setting (i.e., "dominant-view" topics) or has 
two opposing views of similar popularity in that social 
setting (i.e., "mixed-view" topics).

Contrary to our hypotheses, we found that gaze coupling *increased*
when listeners disagreed with the speakers. When performing
exploratory analyses of the unexpected social setting variable (i.e.,
whether the topic was dominant-view or mixed-view), we found
another intriguing effect: Not only did listeners in all mixed-view
(or controversial) topics have greater gaze coordination with speakers,
but listeners who *disagreed* with the speakers about controversial
topics had the *highest* amount of gaze coordination -- *even though the
listeners could not see what the speakers were seeing*. These coordination
dynamics suggest that listeners may be trying to attend more to speakers
who are providing controversial opinions or opinions that are different
from their own; given that previous work has causally linked gaze
coordination with understanding (Richardson & Dale, 2005), this may suggest
that listeners are being more active in their attempts to take these
speakers' perspectives and "see the other side."